# Classifier Builder

This notebook helps for the manual writing of a if/else classification function for template recognition.


In [ ]:
import time
import glob
import numpy
import matplotlib.image
import matplotlib.pyplot
import hoplite.game.terrain

## 1. Groundtruth Dataset

Edit the following cell to load templates into the `groundtruth` list. Each element in this list is a dictionnary with the following keys:
 - `"label"`: Label associated to the template
 - `"part"`: Pixel matrix representation of the template
 - `"filename"`: Source file

In [ ]:
folders = [
    "draft/screenshots/terrain/floor/*.png",
]

groundtruth = list()
for folder in folders:
    for filename in glob.glob(folder):
        image = matplotlib.image.imread(filename)[:, :, :3]
        label = hoplite.game.terrain.SurfaceElement.GROUND
        groundtruth.append({
            "filename": filename,
            "part": image,
            "label": label,
        })

print("Loaded", len(groundtruth), "extracts.")

## 2. Helpers Functions

Not modification are needed.

In [ ]:
def is_close(tgt, ref):
    return numpy.isclose(tgt - ref, 0, atol=.001).all()

def try_classifier(fcn):
    time_start = time.time()
    for i, item in enumerate(groundtruth):
        prediction = fcn(item["part"])
        if prediction != item["label"]:
            print("Item %d -" % (i + 1), item["filename"])
            print("Prediction:", prediction)
            print("Correct label:", item["label"])
            figure = matplotlib.pyplot.figure(figsize=(6, 6))
            ax = figure.add_subplot(1, 1, 1)
            ax.imshow(item["part"])
            ax.set_xlim(0, item["part"].shape[1] - 1)
            ax.set_ylim(item["part"].shape[0] - 1, 0)
            ax.set_xlabel("j")
            ax.set_ylabel("i")
            ax.set_xticks(range(item["part"].shape[1]), minor=True)
            ax.set_yticks(range(item["part"].shape[0]), minor=True)
            ax.grid(which='minor', alpha=0.2)
            ax.grid(which='major', alpha=0.5)
            matplotlib.pyplot.show()
            while True:
                coordinates = input("Enter i,j coordinates> ")
                if coordinates == "":
                    break
                i, j = list(map(lambda x: int(x.strip()), coordinates.split(",")))
                print("if is_close(part[%d, %d], [%f, %f, %f]):"\
                          % (i, j, *item["part"][i, j]))
                print(" " * 8 + "return \"%s\"" % item["label"])
                break
            return False
    print("Success! Elapsed: %.3f seconds" % (time.time() - time_start))
    return True

## 2. Classifier Writing

Execute the following cell, and edit the `classify` function until there is no more assertion error. Your classifier should then be safe.

In [ ]:
def classify(part):
    if is_close(part[26, 26], [0.223529, 0.235294, 0.223529]):
        return hoplite.game.terrain.SurfaceElement.GROUND
    return None

assert try_classifier(classify)